In [1]:
import duckdb
import pandas as pd

crashes_df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv', low_memory = False)
boro_df = pd.read_csv('Borough_Boundaries_20260207.csv', low_memory = False)

In [5]:
# Initialize DuckDB and enable spatial functions
con = duckdb.connect()
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Register pandas DataFrames so they can be queried with SQL
con.register("boroughs_raw", boro_df)
con.register("crashes_raw", crashes_df)

# Create a borough table with geometry for spatial lookups
con.execute("""
CREATE OR REPLACE TABLE boroughs AS
SELECT
  BoroName AS borough,
  ST_GeomFromText(the_geom) AS geom
FROM boroughs_raw;
""")

# Create a cleaned crashes table and normalize missing borough values
con.execute("""
CREATE OR REPLACE TABLE crashes AS
SELECT
  CAST(COLLISION_ID AS BIGINT) AS collision_id,
  TRY_CAST(LATITUDE AS DOUBLE) AS lat,
  TRY_CAST(LONGITUDE AS DOUBLE) AS lon,
  NULLIF(TRIM(CAST(BOROUGH AS VARCHAR)), '') AS borough_reported
FROM crashes_raw;
""")

# Optional sanity check to ensure collision_id is unique
dupes = con.execute("""
SELECT collision_id, COUNT(*) AS n
FROM crashes
GROUP BY collision_id
HAVING COUNT(*) > 1
LIMIT 5;
""").df()

if len(dupes) > 0:
    raise ValueError("collision_id not unique. examples:\n" + dupes.to_string(index = False))

# Isolate crashes missing boroughs but with valid NYC coordinates
con.execute("""
CREATE OR REPLACE TABLE crashes_missing_boro AS
SELECT collision_id, lat, lon
FROM crashes
WHERE borough_reported IS NULL
  AND lat IS NOT NULL AND lon IS NOT NULL
  AND NOT (lat = 0 AND lon = 0)
  AND lat BETWEEN 40.40 AND 40.95
  AND lon BETWEEN -74.30 AND -73.65;
""")

# Spatially assign boroughs using point-in-polygon logic
lookup_df = con.execute("""
SELECT
  c.collision_id,
  b.borough AS borough_derived
FROM crashes_missing_boro c
LEFT JOIN boroughs b
  ON ST_Within(ST_Point(c.lon, c.lat), b.geom);
""").df()

# Export derived borough assignments for downstream joins
lookup_df.to_csv("borough_lookup.csv", index = False)

# Quick validation summary
print(f"wrote borough_lookup.csv rows: {len(lookup_df):,}")
print("derived nulls:", int(lookup_df["borough_derived"].isna().sum()))
print(lookup_df["borough_derived"].value_counts(dropna = False).head(10))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ wrote borough_lookup.csv rows: 473,594
derived nulls: 8424
borough_derived
Queens           156002
Brooklyn         120647
Manhattan         81349
Bronx             73376
Staten Island     33796
None               8424
Name: count, dtype: int64
